In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Wyświetla wszystkie wiersze
pd.set_option('display.max_columns', None)  # Wyświetla wszystkie kolumny
pd.set_option('display.width', 1000)  # Szerszy output, by uniknąć zawijania
pd.options.display.float_format = '{:.2f}'.format

import sys
import os
sys.path.append(os.path.abspath(".."))

import matplotlib.pyplot as plt

from bot.data.historic.binance_client_wrapper import BinanceClientWrapper
from bot.data.historic.binance_data_fetcher import BinanceDataFetcher
from bot.core.utils import Utils

# client = BinanceClientWrapper()
# client.get_klines('BTCUSDC', '1h', 1735689600000, 1740614400000)

fetcher = BinanceDataFetcher()


df_1 = fetcher.fetch_klines_dates('BTCUSDT', '5m', '2025-03-04 00:00:00', '2025-03-04 23:59:59', 0) # "%Y-%m-%d %H:%M:%S"
df_1 = Utils.add_stoch_rsi(df_1)
df_1['timestamp'] = pd.to_datetime(df_1['timestamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
df_1 = df_1.drop(columns=['open', 'high', 'low', 'volume', 'is_closed', 'symbol', 'interval'])

print(df_1);

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_1['timestamp'], df_1['stoch_rsi_k'], label='Stoch RSI K', color='b')

# Formatowanie wykresu
plt.xticks(df_1['timestamp'][::20], rotation=45)

plt.xlabel('Timestamp')
plt.ylabel('Stochastic RSI K')
plt.title('Stochastic RSI K over Time')
plt.legend()
plt.xticks(rotation=45)  # Rotacja etykiet na osi X
plt.grid(True)

y_min, y_max = df_1['stoch_rsi_k'].min(), df_1['stoch_rsi_k'].max()
plt.ylim(y_min, y_max * 4)

# Wyświetlenie wykresu
plt.show()

In [ ]:
df_2 = fetcher.fetch_klines_dates('BTCUSDT', '1m', '2025-03-04 00:00:00', '2025-03-04 23:59:59', 0) # "%Y-%m-%d %H:%M:%S"
df_2 = Utils.add_stoch_rsi(df_2)
df_2['timestamp'] = pd.to_datetime(df_2['timestamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
df_2 = df_2.drop(columns=['open', 'high', 'low', 'volume', 'is_closed', 'symbol', 'interval'])

print(df_2);

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_2['timestamp'], df_2['stoch_rsi_k'], label='Stoch RSI K', color='b')

# Formatowanie wykresu
plt.xticks(df_2['timestamp'][::20], rotation=45)

plt.xlabel('Timestamp')
plt.ylabel('Stochastic RSI K')
plt.title('Stochastic RSI K over Time')
plt.legend()
plt.xticks(rotation=45)  # Rotacja etykiet na osi X
plt.grid(True)

y_min, y_max = df_2['stoch_rsi_k'].min(), df_2['stoch_rsi_k'].max()
plt.ylim(y_min, y_max * 4)

# Wyświetlenie wykresu
plt.show()

In [ ]:
df_1

In [ ]:
df_1 = fetcher.fetch_klines_dates('BTCUSDT', '5m', '2025-03-04 00:00:00', '2025-03-04 23:59:59', 0) # "%Y-%m-%d %H:%M:%S"
df_1['timestamp'] = pd.to_datetime(df_1['timestamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
df_1 = df_1[['timestamp', 'close']]
df_1 = Utils.add_stoch_rsi(df_1)
df_1

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Rysowanie linii dla Stoch RSI K
plt.plot(df_1['timestamp'], df_1['stoch_rsi_k'], label='Stoch RSI K', color='b')

# Rysowanie linii dla Stoch RSI D
plt.plot(df_1['timestamp'], df_1['stoch_rsi_d'], label='Stoch RSI D', color='r')

# Formatowanie wykresu
plt.xticks(df_1['timestamp'][::20], rotation=45)
plt.xlabel('Timestamp')
plt.ylabel('Stochastic RSI')
plt.title('Stochastic RSI K & D over Time')
plt.legend()
plt.grid(True)

# Ustawienie zakresu dla osi Y
y_min = min(df_1['stoch_rsi_k'].min(), df_1['stoch_rsi_d'].min())
y_max = max(df_1['stoch_rsi_k'].max(), df_1['stoch_rsi_d'].max())
plt.ylim(y_min, y_max * 1.1)  # Dodanie lekkiego marginesu

# Wyświetlenie wykresu
plt.show()


In [ ]:
import numpy as np

# Tworzymy nową kolumnę crossover, domyślnie 0
df['crossover'] = 0

# Warunki przecięcia
for i in range(1, len(df)):
    if not np.isnan(df.loc[i, 'stoch_rsi_k']) and not np.isnan(df.loc[i, 'stoch_rsi_d']):
        prev_k = df.loc[i - 1, 'stoch_rsi_k']
        prev_d = df.loc[i - 1, 'stoch_rsi_d']
        curr_k = df.loc[i, 'stoch_rsi_k']
        curr_d = df.loc[i, 'stoch_rsi_d']
        
        # Przecięcie od dołu (sygnał kupna)
        if prev_k < prev_d and curr_k > curr_d:
            df.loc[i, 'crossover'] = 1
        
        # Przecięcie od góry (sygnał sprzedaży)
        elif prev_k > prev_d and curr_k < curr_d:
            df.loc[i, 'crossover'] = -1

In [ ]:
df